In [21]:
import numpy as np
import cv2
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import math
import open3d as o3d
import matplotlib.pyplot as plt
from sklearn.linear_model import RANSACRegressor
from sklearn.cluster import KMeans
from scipy.optimize import minimize
from pathlib import Path
from plyfile import PlyData
from collections import Counter


In [22]:
def calculate_normal_vector(points):
    pca = PCA(n_components=3)
    pca.fit(points)
    normal_vector = pca.components_[-1]  # The normal vector is the eigenvector with the smallest eigenvalue

    normal_vector = np.array(normal_vector)
    angle_with_x_axis = np.arccos(np.dot(normal_vector, [1, 0, 0]))
    angle_with_y_axis = np.arccos(np.dot(normal_vector, [0, 1, 0]))
    angle_with_z_axis = np.arccos(np.dot(normal_vector, [0, 0, 1]))
    normal_vector = [angle_with_x_axis, angle_with_y_axis, angle_with_z_axis]
    # print(normal_vector)
    return normal_vector

def angle_with_z_axis(vector):
    vector = vector / np.linalg.norm(vector)
    return np.arccos(np.dot(vector, [0, 0, -1]))

def xy2radius(x, y):
    return math.sqrt(x**2 + y**2)

def xy2theta(x, y):
    return math.atan2(y, x)

In [83]:
class Zone:
    def __init__(self):
        self.points = []

class CZM:
    def __init__(self, ply_file_path):
        self.ply_file_path = ply_file_path
        self.ply_file_name = os.path.splitext(os.path.basename(ply_file_path))[0]  # 'frame002846-1581624937_362' 같은 형태로 파일 이름을 저장

        self.point_cloud = None

        # Define range parameters
        self.min_range_ = 4
        self.min_range_z2_ = 12
        self.min_range_z3_ = 28
        self.min_range_z4_ = 60
        self.max_range_ = 100.0
        self.min_ranges_ = [self.min_range_, self.min_range_z2_, self.min_range_z3_, self.min_range_z4_]

        # Define zone, sector, and ring parameters
        self.num_zones = 4
        self.num_sectors_each_zone = [72, 48, 48, 64]
        self.num_rings_each_zone = [4, 4, 5, 5]
        self.ring_sizes_ = [
            (self.min_range_z2_ - self.min_range_) / self.num_rings_each_zone[0],
            (self.min_range_z3_ - self.min_range_z2_) / self.num_rings_each_zone[1],
            (self.min_range_z4_ - self.min_range_z3_) / self.num_rings_each_zone[2],
            (self.max_range_ - self.min_range_z4_) / self.num_rings_each_zone[3]
        ]
        self.sector_sizes_ = [
            2 * math.pi / self.num_sectors_each_zone[0],
            2 * math.pi / self.num_sectors_each_zone[1],
            2 * math.pi / self.num_sectors_each_zone[2],
            2 * math.pi / self.num_sectors_each_zone[3]
        ]
        
        self.images_to_cluster = []
        self.labels = []
        self.max_stripes_list = []

        self.cluster_labels = {
            "cluster_0": [3, 23],
            "cluster_1": [6, 31, 33, 19, 34],
            "cluster_2": [4, 17, 8, 18, 27]
        }

        # Load point cloud
        self.load_point_cloud()
        
    def read_ply(self, filename):
        """ read XYZ and label from filename PLY file """
        plydata = PlyData.read(filename)
        pc = plydata['vertex'].data
        pc_array = np.array([[x, y, z, label] for x, y, z, _, _, _, _, _, _, label, _, _, _ in pc])
        return pc_array

    def load_point_cloud(self):
        self.point_cloud = self.read_ply(self.ply_file_path)
        
        # z_values = [pt[2] for pt in self.point_cloud]
        # self.min_z = min(z_values)
        # self.max_z = max(z_values)
        # print(self.min_z, self.max_z)
        
    def calculate_normal_vector(self, points):
        pca = PCA(n_components=3)
        pca.fit(points)
        normal_vector = pca.components_[-1]

        normal_vector = np.array(normal_vector)
        angle_with_x_axis = np.arccos(np.dot(normal_vector, [1, 0, 0]))
        angle_with_y_axis = np.arccos(np.dot(normal_vector, [0, 1, 0]))
        angle_with_z_axis = np.arccos(np.dot(normal_vector, [0, 0, 1]))
        normal_vector = [angle_with_x_axis, angle_with_y_axis, angle_with_z_axis]
        # print(normal_vector)
        return normal_vector

    def angle_with_z_axis(self, vector):
        vector = vector / np.linalg.norm(vector)
        return np.arccos(np.dot(vector, [0, 0, -1]))
    


    def segment_czm(self):
        ring_sizes_ = [(self.min_range_z2_ - self.min_range_) / self.num_rings_each_zone[0],
                    (self.min_range_z3_ - self.min_range_z2_) / self.num_rings_each_zone[1],
                    (self.min_range_z4_ - self.min_range_z3_) / self.num_rings_each_zone[2],
                    (self.max_range_ - self.min_range_z4_) / self.num_rings_each_zone[3]]

        sector_sizes_ = [2 * math.pi / self.num_sectors_each_zone[0], 
                        2 * math.pi / self.num_sectors_each_zone[1], 
                        2 * math.pi / self.num_sectors_each_zone[2], 
                        2 * math.pi / self.num_sectors_each_zone[3]]

        sector_indices = [] 

        czm = [[
            [Zone() for _ in range(self.num_sectors_each_zone[zone_idx])]
            for _ in range(self.num_rings_each_zone[zone_idx])
        ] for zone_idx in range(self.num_zones)]

        for pt in self.point_cloud:
            x, y, z, label = pt
            r = xy2radius(x, y)
            if r <= self.max_range_ and r > self.min_range_:
                theta = xy2theta(x, y)
                zone_idx = ring_idx = sector_idx = 0

                if r < self.min_range_z2_:
                    zone_idx = 0
                    ring_idx = min(int((r - self.min_range_) / ring_sizes_[0]), self.num_rings_each_zone[0]-1)
                    sector_idx = min(int(theta / sector_sizes_[0]), self.num_sectors_each_zone[0]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, label])
                elif r < self.min_range_z3_:
                    zone_idx = 1
                    ring_idx = min(int((r - self.min_range_z2_) / ring_sizes_[1]), self.num_rings_each_zone[1]-1)
                    sector_idx = min(int(theta / sector_sizes_[1]), self.num_sectors_each_zone[1]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, label])
                elif r < self.min_range_z4_:
                    zone_idx = 2
                    ring_idx = min(int((r - self.min_range_z3_) / ring_sizes_[2]), self.num_rings_each_zone[2]-1)
                    sector_idx = min(int(theta / sector_sizes_[2]), self.num_sectors_each_zone[2]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, label])
                else:
                    zone_idx = 3
                    ring_idx = min(int((r - self.min_range_z4_) / ring_sizes_[3]), self.num_rings_each_zone[3]-1)
                    sector_idx = min(int(theta / sector_sizes_[3]), self.num_sectors_each_zone[3]-1)
                    czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, label])
                    
                # czm[zone_idx][ring_idx][sector_idx].points.append([x, y, z, label])
                sector_indices.append((zone_idx, ring_idx, sector_idx)) # 인덱스 추가
        return czm, sector_indices
        #sector_indices 는 각 포인트마다 몇 번째 섹터에 속하는지 나타냄. 

    def process_sectors(self, sectors, img_dim=15, MIN_POINTS=10, MAX_Z_DIFF=0.8): # MAX_Z_DIFF 파라미터 추가
        images = []
        traversable_points = []
        all_image_vectors = []
        image_sector_indices = []

        label_counter = []
        for zone_idx, layer in enumerate(sectors):
            for ring_idx, ring in enumerate(layer):
                for sector_idx, sector in enumerate(ring):
                    if len(sector.points) > MIN_POINTS:

                        points = np.array([pt[:3] for pt in sector.points])
                        normal_vector = calculate_normal_vector(points)

                        sector_center_x = np.mean(points[:, 0])
                        sector_center_y = np.mean(points[:, 1])
                        sector_center_z = np.mean(points[:, 2])

                        z_values = [pt[2] for pt in sector.points]
                        min_z, max_z = min(z_values), max(z_values)

                        # z 좌표의 최대값과 최소값의 차이가 일정 이하인 경우만 처리
                        # if angle_with_z_axis(normal_vector) >= 1.1 and angle_with_z_axis(normal_vector) <= 1.9 and (max_z - min_z) <= MAX_Z_DIFF:
                        traversable_points.extend(points)
                        image = np.zeros((img_dim, img_dim), dtype=np.uint8)

                        x_values = [pt[0] for pt in sector.points]
                        y_values = [pt[1] for pt in sector.points]

                        min_x, max_x = min(x_values), max(x_values)
                        min_y, max_y = min(y_values), max(y_values)

                        for pt in sector.points:
                            normalized_x = int((pt[0] - min_x) / (max_x - min_x) * (img_dim - 1))
                            normalized_y = int((pt[1] - min_y) / (max_y - min_y) * (img_dim - 1))
                            normalized_z = int((pt[2] - min_z) / (max_z - min_z) * 255)

                            image[normalized_y, normalized_x] = normalized_z

                        image_2d = image.reshape(img_dim, img_dim)

                        # 가장 빈도수가 높은 라벨 결정
                        labels = [pt[3] for pt in sector.points]
                        # label_frequencies = Counter(labels)
                        # print("Label Frequencies:", label_frequencies)
                        most_common_label = Counter(labels).most_common(1)[0][0]
                        label_counter.append(most_common_label)
                        
                        # print(most_common_label)
                        
                        # 라벨 기반으로 폴더 이름 결정
                        folder_name = "etc"
                        for cluster_name, label_set in self.cluster_labels.items():
                            if most_common_label in label_set:
                                folder_name = cluster_name
                                break


                        # 해당 폴더가 존재하지 않는 경우 폴더 생성
                        if not os.path.exists(folder_name):
                            os.makedirs(folder_name)

                        img_path = os.path.join(folder_name, f"{self.ply_file_name}_image_zone_{zone_idx}_ring_{ring_idx}_sector_{sector_idx}.png")
                        cv2.imwrite(img_path, image_2d)
                        
                        images.append(image_2d)
                        image_sector_indices.append((zone_idx, ring_idx, sector_idx)) # 해당 이미지의 sector 인덱스 추가

                        image_vector = image_2d.reshape(-1, 1)
                        all_image_vectors.append(image_vector)
        # points_pcd = o3d.geometry.PointCloud()
        # points_pcd.points = o3d.utility.Vector3dVector(traversable_points)
        # pcd_path = "./traversable_points_pcd.pcd"
        # o3d.io.write_point_cloud(pcd_path, points_pcd)

        return images, image_sector_indices, label_counter

In [84]:
ply_file_path = "/media/rtlink/JetsonSSD-256/Download/Rellis_3D_os1_cloud_node_color_ply/Rellis-3D/00000/os1_cloud_node_color_ply/ply_files/frame002846-1581624937_362.ply"
czm = CZM(ply_file_path) 

In [85]:
sectors, sector_indices = czm.segment_czm() 

In [86]:
images, image_sector_indices, label_counter = czm.process_sectors(sectors) # 인스턴스 czm을 통해 메서드를 호출하고, image_sector_indices도 받습니다.

In [87]:
print(label_counter)

[3.0, 3.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 17.0, 3.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 3.0, 19.0, 3.0, 3.0, 3.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 3.0, 3.0, 3.0, 3.0, 3.0,

In [88]:
label_frequencies = Counter(label_counter)
print("Label Frequencies:", label_frequencies)

Label Frequencies: Counter({19.0: 256, 4.0: 100, 3.0: 54, 17.0: 1})


In [ ]:

base_folder = "/media/rtlink/JetsonSSD-256/Download/Rellis_3D_os1_cloud_node_color_ply/Rellis-3D/00000/os1_cloud_node_color_ply/ply_files/"

# base_folder 내의 모든 .ply 파일을 가져옴
ply_files = [f for f in os.listdir(base_folder) if f.endswith('.ply')]

for ply_file in ply_files:
    ply_file_path = os.path.join(base_folder, ply_file)
    
    czm = CZM(ply_file_path)
    sectors, sector_indices = czm.segment_czm()
    images, image_sector_indices, label_counter = czm.process_sectors(sectors)


In [91]:
import random
import shutil

def balance_clusters(data_directory):
    # 폴더(클러스터) 목록 가져오기
    cluster_folders = [f for f in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, f))]
    
    # 각 클러스터의 이미지 수 확인
    cluster_sizes = {}
    for folder in cluster_folders:
        cluster_sizes[folder] = len(os.listdir(os.path.join(data_directory, folder)))
    
    # 가장 작은 클러스터의 이미지 수 확인
    min_size = min(cluster_sizes.values())
    
    # 각 클러스터에서 무작위로 이미지 뽑기
    for folder in cluster_folders:
        current_folder_path = os.path.join(data_directory, folder)
        current_images = os.listdir(current_folder_path)
        random.shuffle(current_images)  # 이미지 목록 섞기
        
        # 필요한 이미지만큼만 남기고 나머지는 삭제
        for img in current_images[min_size:]:
            os.remove(os.path.join(current_folder_path, img))



In [92]:
balance_clusters('/home/rtlink/jiwon/paper_ws/src/bumpypatch/static scene/validation_dataset')

In [93]:
base_folder = "/media/rtlink/JetsonSSD-256/Download/Rellis_3D_os1_cloud_node_color_ply/Rellis-3D/00000/os1_cloud_node_color_ply/ply_files/"
# base_folder 내의 모든 .ply 파일을 가져옴
ply_files = [f for f in os.listdir(base_folder) if f.endswith('.ply')]

# 마지막 100개의 .ply 파일만 사용
ply_files = ply_files[-100:]

for ply_file in ply_files:
    ply_file_path = os.path.join(base_folder, ply_file)
    
    czm = CZM(ply_file_path)
    sectors, sector_indices = czm.segment_czm()
    images, image_sector_indices, label_counter = czm.process_sectors(sectors)


KeyboardInterrupt: 